In [ ]:
def sep_base (ids, mask, min, max):
    with torch.inference_mode():
        last_hidden_states = model(ids[min : max], attention_mask=mask[min : max])
    return last_hidden_states[0][:,0,:].numpy()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Скачиваем необходимые библиотеки

In [2]:
!pip install sentencepiece 

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 636 kB 49.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 10.3 MB/s 


# Функции обработки текста

In [5]:
def clean_lemm_general(text_general):
    '''Основной препроцессинг текста. Проводим основную лемматизацию'''
    import nltk
    from nltk.stem import WordNetLemmatizer
    import pandas as pd
    import numpy as np
    from pymorphy2 import MorphAnalyzer
    nltk.download('wordnet')
    def clean(text):
        '''Чистим текст от мусора'''
        import re
        re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
        text_clean_pre = text.lower() # приводим все символы к нижнему регистру
        text_clean_pre = re.sub(r'\d+',' ', text_clean_pre) # удаляем все числа 
        text_clean_pre = re.sub(r'[^\w\s]','', text_clean_pre) # удаляем все знаки припенания
        text_clean_pre = re.sub(r'http\S+', " ", text_clean_pre)
        text_clean_pre = re.sub(r'@\w+',' ',text_clean_pre)
        text_clean_pre = re.sub(r'#\w+', ' ', text_clean_pre)
        text_clean_pre = re_br.sub("\n", text_clean_pre)
        return text_clean_pre

    text_clean_gen = text_general.apply(clean)

    morph = MorphAnalyzer()
    text_lem = WordNetLemmatizer()
    morph_text = []
    lem_text = []
    for words in text_clean_gen:
        morph_text.append(' '.join([morph.parse(word)[0].normal_form for word in words.split()]))
    for words in morph_text:
        lem_text.append(' '.join([text_lem.lemmatize(word) for word in words.split()]))
    return lem_text

def clean_lemm_test(text):
    '''Препроцессинг текста. Проводим лемматизацию'''
    import re
    import nltk
    from nltk.stem import WordNetLemmatizer
    from pymorphy2 import MorphAnalyzer
    nltk.download('wordnet')
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    text_clean_test = text.lower() # приводим все символы к нижнему регистру
    text_clean_test = re.sub(r'\d+', ' ', text_clean_test) # удаляем все числа 
    text_clean_test = re.sub(r'[^\w\s]','', text_clean_test) # удаляем все знаки припенания
    text_clean_test = re.sub(r'http\S+', " ", text_clean_test)
    text_clean_test = re.sub(r'@\w+',' ',text_clean_test)
    text_clean_test = re.sub(r'#\w+', ' ', text_clean_test)
    text_clean_pre = re_br.sub("\n", text_clean_test)

    morph = MorphAnalyzer()
    text_lem= WordNetLemmatizer()
    morph_text = []
    lem_text = []
    morph_text.append(' '.join([morph.parse(words)[0].normal_form for words in text_clean_pre.split()]))
    morph_text = ''.join(morph_text)
    lem_text.append(' '.join([text_lem.lemmatize(words) for words in morph_text.split()]))
    return lem_text

def token(lem_text):
    '''Проводим токенизацию текста'''
    import nltk
    from nltk.tokenize import RegexpTokenizer
    reg_tokenizer = RegexpTokenizer('\w+') # берем только слова
    tokenized_text = reg_tokenizer.tokenize_sents(lem_text)
    return tokenized_text

def nltk_stop_words(tokenized_text):
    '''Удаляем из текста стоп-слова 1 вериант (медленнее)'''
    import nltk
    from nltk.corpus import stopwords # стопслова
    nltk.download('stopwords')
    sw_rus = stopwords.words('russian') # список стоп-слов 
    sw_eng = stopwords.words('english')
    clean_stop_words = [] 
    for i, element in enumerate(tokenized_text):
        clean_stop_words.append(' '.join([word for word in element if word not in sw_rus and word not in sw_eng]))
    return clean_stop_words

def gens(tokenized_text):
    '''Удаляем из текста стоп-слова 2 вариант (более быстрый)'''
    import gensim
    sw2 = gensim.parsing.preprocessing.STOPWORDS
    clean_tokenized_reviews2 = []
    for i, element in enumerate(tokenized_text):
        clean_tokenized_reviews2.append(' '.join([word for word in element if word not in sw2]))
    return clean_tokenized_reviews2

# Функции парсинга сайта 'КиноПоиск'

In [7]:
def load_data(url):
    r = requests.get(url, headers = {
          'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
          'Accept-Language' : 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,en-GB;q=0.6',
          'Accept' : '*/*'
     }) # отправка http запроса
    soup = BeautifulSoup(r.text, 'html.parser')# создание html парсера
    reviews = soup.find_all(class_='_reachbanner_')# сохранение только отзывов
    username = soup.find_all('a', attrs={'itemprop' : 'name'})
    reviews_clean = []
    for review in reviews:# очистка от лишней html разметки
        reviews_clean.append(review.find_all(text=True))
    return reviews_clean, username

def convert(reviews, users): # преобразование отзывов в строки
    review_converted = []
    user_converted = []
    for review in reviews:
        for i in review:
            map(str, i)
        review = ''.join(review)
        review_converted.append(review)

    for us in users:
        map(str, us.text)
        new_user = us.text.replace(" ","")
        user = ''.join(new_user)
        user_converted.append(user)
    return review_converted, user_converted

def parsing(url, status, path):
    page = 1
    delays = [11, 12, 13, 11.5, 12.5, 13.5, 11.2, 12.3, 11.8]
    time.sleep(np.random.choice(delays)) # Выбор рандомной задержки
    films_pars = {
         'https://www.kinopoisk.ru/film/41519/':'Брат',
         'https://www.kinopoisk.ru/film/468196/':'9 рота',
         'https://www.kinopoisk.ru/film/84674/':'Сталинград',
         }
    while True:
        loaded_data, user_data = load_data(url + 'reviews/ord/date/status/{}/perpage/75/page/{}/'.format(status, page))
        if loaded_data == []:
            break
        else:
            # Eсли папок с классами не существует, программа их создаст
            if not os.path.exists(path + r'{}'.format(status)):
                os.makedirs(path + r'{}'.format(status))
            converted_data, user_convert_data = convert(loaded_data, user_data)
            # Сохранение отзывов
            for i, review in enumerate(converted_data):
                with open(path + r'{}/{}_{}_{}.txt'.format(status,films_pars[url], page, i), 'w', encoding = 'utf-8') as output:
                    output.write(user_convert_data[i] +' '+ review)
            page += 1
            time.sleep(np.random.choice(delays))

# Устанавливаем библиотеки

In [92]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import transformers as ppb
import warnings
from xgboost import XGBClassifier
import requests
from bs4 import BeautifulSoup
import time
import os
import textwrap
from transformers import GPT2LMHeadModel, AdamW
from transformers import GPT2Tokenizer
import re
import random
from tqdm.notebook import tqdm
import tensorflow as tf
warnings.filterwarnings('ignore')

# Парсим сайт 'КиноПоиск'

In [9]:
path = '/content/drive/MyDrive/Elbrus/Films/'
films = {
         'https://www.kinopoisk.ru/film/41519/':'"Брат" 1997',
         'https://www.kinopoisk.ru/film/84674/':'"9 рота" 2005',
         'https://www.kinopoisk.ru/film/468196/':'"Сталинград" 2013',
         }
urles = ['https://www.kinopoisk.ru/film/41519/',
         'https://www.kinopoisk.ru/film/84674/',
         'https://www.kinopoisk.ru/film/468196/',
statuses = ['good', 'bad','neutral']
delays = [15, 20, 13, 18, 12.5, 13.5, 25, 12.3, 23]
for url in urles:
    print(films[url])
    for status in statuses:
        try:
            parsing(url = url, status = status, path=path)
            print(f'{status} category done')
            time.sleep(np.random.choice(delays))
        except AttributeError:
            print('Бан получен: {}, {}'.format(url, status))
            break
    else:
        print('one url done')
        continue
    break

"Брат" 1997
good category done
bad category done
neutral category done
one url done
"9 рота" 2005
good category done
bad category done
neutral category done
one url done
"Сталинград" 2013
good category done
bad category done
neutral category done
one url done
"Защитники" 2017
good category done
bad category done
neutral category done
one url done
"Холоп" 2019
good category done
bad category done
neutral category done
one url done


# Обрабатываем полученные данные в один DataFrame

In [15]:
name_films = ['9 рота', 'Брат', 'Сталинград']
bad = []
for film in name_films:
    if film == '9 рота':
        for i in range(1,4):
            if i != 3:
                for j in range(75):
                    bd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/bad/{film}_{i}_{j}.txt', header=None, sep='\n')
                    bd.rename({0:'Review'},axis=1, inplace=True)
                    bad.append(' '.join(bd.Review))
            else:
                for k in range(69):
                    bd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/bad/{film}_{i}_{k}.txt', header=None, sep='\n')
                    bd.rename({0:'Review'},axis=1, inplace=True)
                    bad.append(' '.join(bd.Review))

    if film == 'Брат':
        for j in range(17):
            bd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/bad/{film}_1_{j}.txt', header=None, sep='\n')
            bd.rename({0:'Review'},axis=1, inplace=True)
            bad.append(' '.join(bd.Review))

    if film == 'Сталинград':
        for j in range(20):
            bd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/bad/{film}_1_{j}.txt', header=None, sep='\n')
            bd.rename({0:'Review'},axis=1, inplace=True)
            bad.append(' '.join(bd.Review))  
df_bad = pd.DataFrame(bad)
df_bad.rename({0:'Review'}, axis=1, inplace=True)
df_bad['Sentiment'] = 'bad'

neutral = []
for film in name_films:
    if film == '9 рота':
        for i in range(1,3):
            if i == 1:
                for j in range(75):
                    neu = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/neutral/{film}_{i}_{j}.txt', header=None, sep='\n')
                    neu.rename({0:'Review'},axis=1, inplace=True)
                    neutral.append(' '.join(neu.Review))
            else:
                for k in range(42):
                    neu = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/neutral/{film}_{i}_{k}.txt', header=None, sep='\n')
                    neu.rename({0:'Review'},axis=1, inplace=True)
                    neutral.append(' '.join(neu.Review))  
    if film == 'Брат':
        for j in range(25):
            neu = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/neutral/{film}_1_{j}.txt', header=None, sep='\n')
            neu.rename({0:'Review'},axis=1, inplace=True)
            neutral.append(' '.join(neu.Review))
    if film == 'Сталинград':
        break
df_neu = pd.DataFrame(neutral)
df_neu.rename({0:'Review'}, axis=1, inplace=True)
df_neu['Sentiment'] = 'neutral'

good = []
for film in name_films:
    if film == '9 рота':
        for i in range(1,4):
            if i != 3:
                for j in range(75):
                    gd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/good/{film}_{i}_{j}.txt', header=None, sep='\n')
                    gd.rename({0:'Review'},axis=1, inplace=True)
                    good.append(' '.join(gd.Review))
            else:
                for k in range(63):
                    gd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/good/{film}_{i}_{k}.txt', header=None, sep='\n')
                    gd.rename({0:'Review'},axis=1, inplace=True)
                    good.append(' '.join(gd.Review))
    if film == 'Брат':
        for i in range(1,3):
            if i == 1:
                for j in range(75):
                    gd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/good/{film}_{i}_{j}.txt', header=None, sep='\n')
                    gd.rename({0:'Review'},axis=1, inplace=True)
                    good.append(' '.join(gd.Review))
            else:
                for k in range(48):
                    gd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/good/{film}_{i}_{k}.txt', header=None, sep='\n')
                    gd.rename({0:'Review'},axis=1, inplace=True)
                    good.append(' '.join(gd.Review)) 
    if film == 'Сталинград':
        for j in range(45):
            gd = pd.read_table(f'/content/drive/MyDrive/Elbrus/Films/good/{film}_1_{j}.txt', header=None, sep='\n')
            gd.rename({0:'Review'},axis=1, inplace=True)
            good.append(' '.join(gd.Review))              
df_good = pd.DataFrame(good)
df_good.rename({0:'Review'}, axis=1, inplace=True)
df_good['Sentiment'] = 'positive'

df_all = pd.concat([df_bad, df_neu, df_good])
df_all = df_all.sample(frac=1).reset_index(drop=True)
df_all

,Review,Sentiment
0,Vitaliy23 Снимавшийся под Петербургом (поселок...,bad
1,"Эстет В нашей стране, пожалуй нет недостатка в...",neutral
2,"newht Сказать, что фильм мне не понравился - с...",bad
3,wasserleiter За что многие ругают этот фильм? ...,positive
4,Druzhee Моя рецензия будет очень краткой. Любл...,neutral
...,...,...
774,"kinocor Надо начать с того, что солдаты Красно...",bad
775,"kingfars Какая была 9 рота! Военная драма, кот...",positive
776,"AnnArhia_ Для тех, кто Впервые я посмотрела фи...",positive
777,ДонВитоКорлеоне Впечатления от просмотра фильм...,neutral


In [16]:
print(df_all.Sentiment.value_counts())
df_all.shape

positive    381
bad         256
neutral     142
Name: Sentiment, dtype: int64


(779, 2)

# Применяем LableEncoding

In [23]:
df_all['Sentiment_encode'] = LabelEncoder().fit_transform(df_all['Sentiment'])
name = []
for rev in range(df_all.shape[0]):
    name.append(df_all.Review[rev].split(' ')[0])
df_all['User_names'] = name
df_all

,Review,Sentiment,Sentiment_encode,User_names
0,Vitaliy23 Снимавшийся под Петербургом (поселок...,bad,0,Vitaliy23
1,"Эстет В нашей стране, пожалуй нет недостатка в...",neutral,1,Эстет
2,"newht Сказать, что фильм мне не понравился - с...",bad,0,newht
3,wasserleiter За что многие ругают этот фильм? ...,positive,2,wasserleiter
4,Druzhee Моя рецензия будет очень краткой. Любл...,neutral,1,Druzhee
...,...,...,...,...
774,"kinocor Надо начать с того, что солдаты Красно...",bad,0,kinocor
775,"kingfars Какая была 9 рота! Военная драма, кот...",positive,2,kingfars
776,"AnnArhia_ Для тех, кто Впервые я посмотрела фи...",positive,2,AnnArhia_
777,ДонВитоКорлеоне Впечатления от просмотра фильм...,neutral,1,ДонВитоКорлеоне


# Чистим полученные рецензии

In [25]:
review = clean_lemm_general(df_all['Review'])
review = token(review)
review = nltk_stop_words(review)
df_all['Review_new'] = review
df_all.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review,Sentiment,Sentiment_encode,User_names,Review_new
0,Vitaliy23 Снимавшийся под Петербургом (поселок...,bad,0,Vitaliy23,vitaliy сниматься петербург посёлок сапёрный л...
1,"Эстет В нашей стране, пожалуй нет недостатка в...",neutral,1,Эстет,эстет наш страна пожалуй недостаток работа тем...
2,"newht Сказать, что фильм мне не понравился - с...",bad,0,newht,newht сказать фильм понравиться самый гуманный...
3,wasserleiter За что многие ругают этот фильм? ...,positive,2,wasserleiter,wasserleiter многие ругать фильм перегрузить з...
4,Druzhee Моя рецензия будет очень краткой. Любл...,neutral,1,Druzhee,druzhee рецензия очень краткий любить фильм во...


# Соединяем все рециензии в один текст

In [312]:
new_text = ' '.join(df_all.Review)
new_text

"Vitaliy23 Снимавшийся под Петербургом (поселок Саперный, Ленинградская область) фильм Федора Бондарчука, претендует стать лучшим фильмом уходящего года, в России. Безусловно у него это выйдет, хотя придется потягаться с еще одним хитом в лице 'Легенды номер 17'. Однако военная тематика нашему зрителю будет более близка.  Я, разумеется, отношусь к тем людям, для кого подвиг наших предков - это что-то необъяснимое. Та воля, те подвиги что они совершали, бывает не понимаешь откуда у людей брались силы для того чтобы выстоять перед захватчиком. Те позывы, которые в очередной раз произносятся в фильме (голос за кадром, как мне показалось именно Бондарчука), взывают зрителя не забывать подвиги и людей, благодаря которым мы с вами сейчас живем.  На этом плюсы картины можно назвать исчерпанными.  Хотя стоит отметить еще один плюс - визуальная составляющая. Картинка красивая, особенная цветовая гамма, которая давит своей тусклостью. Тусклость в хорошем плане. Тусклость, которая словно переноси

In [313]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('You got a GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
You got a GPU: Tesla K80


# Воспользуемся gpt2 от Сбера

In [314]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
text = re.sub('\n{2,}', '\n', new_text)
print(text[:100])

Vitaliy23 Снимавшийся под Петербургом (поселок Саперный, Ленинградская область) фильм Федора Бондарч


In [315]:
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

488081


array([   58,  8787, 31225,  6091,   385,   724,  5229,   424,  5909,
        3924])

In [316]:
tokens = np.array(tokens)
l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

488081 390456 97614


In [317]:
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model_init.to(device);

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

In [318]:
batch_size = 8
max_len = 256
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

total_steps = n_train * epochs
scheduler = ppb.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

190 47


In [78]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    print('epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

epoch 1/5 : training


  0%|          | 0/190 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/47 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/190 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/47 [00:00<?, ?it/s]

In [325]:
prompt = 'ИванИванов'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=120,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=5,
    num_return_sequences=2,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ИванИванов "Звери" "Роковой круг"-"Коппобэт-2011-Впервые..."+19 ноября 2011....(Дарья)...Жена...  Она любит Ивана, И
если честно то мне страшно что он вернется((   ИВАНИЛ (ВАР.) Ты ведь видишь я тебя больше...(Н.) Да мне так кажется и
тебе интересно мое "Одно" "И "Чипирование-Опуск" не зря я его написал!  Правда как будто его "Связи"-среди них.   АНД
------------------
ИванИванов : Мы очень переживаем за эту тему   Александр Александрович : Как правильно действовать.  Необходимо все
сделать самому, и к чему вы можете приложить дополнительные полномочия и ресурсы для укрепления позиции церкви среди
всего народа, не боясь санкций Запада". (С) Иван Иванов 19 августа   Патриарх : Мы все в мире, но что еще есть?  Когда
начнется эта деятельность, чтобы люди говорили, чего не делается, почему у власти люди нет?!  "Нет, да... есть... вот
только, где?"..."Сказания Патриарха..." "...считать людей на Украине безвизом?.. ну
------------------
